In [ ]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  
  입력(inputs) 및 타겟(targets) 데이터는 int형 list이다.
  hprev는 초기 hidden state의 Hx1 배열.
  loss와 gradient, hidden state를 return 한다.
  
  dWxh, dWhh, dWhy, dbh, dby -> gradient인듯.
  
  https://pythonkim.tistory.com/57 -> 그림으로 설명 잘 되어 있음
  
  """
  xs, hs, ys, ps = {}, {}, {}, {} # hs -> hidden state인듯.
  hs[-1] = np.copy(hprev) # hprev를 hidden state에 copy.
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # 1 of k 부호화 -> 해당되는 클래스만 1로 표시 나머지 0
    xs[t][inputs[t]] = 1 
    # depth==2인 xs[t]
    # xs[0]에는 input[0], xs[1]에는 inputs[1] .. xs[t]에는 inputs[n]을 저장.
    
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)  # hidden state 
    # tanh() 호출한거는 vanila RNN 원래 공식인듯.
    # h_t = f_w(h_t-1,x_t) -> rnn 공식 
    # f_w는 some function, parameters == w
    # hs[t] == new state
    # hs[t-1] == old state
    # xs[t] == input vector
    # ==> hs[t]를 바로 직전의 state인 hs[t-1]과
    # 현재 input인 xs[t]를 이용하여 update 한다고 보면 될 듯.
    
    # ↓↓↓↓↓ 그림으로 표현 ↓↓↓↓↓
    
    # h_0   h_1   h_2   h_3   ...   h_t 
    #  A  -> A  -> A  -> A    ... -> A    
    # x_0   x_1   x_2   x_3   ...   x_t      
    
    ys[t] = np.dot(Why, hs[t]) + by # 다음 문자에 대한 정규화되지 않은 로그 확률
    
    # RNN은 step을 거칠 때마다 어떤 결과를 예측하게 된다.
    # 그리고 이런 예측 값을 y라고 한다.
    # 위에 주석으로 그린 그림에서 한 단계마다 y가 나온다고 보면 된다.
    # 여러 hidden state들을 통합적으로 하나의 rnn이라고 보면,
    # 특정 시점 t에서의 예측 값인 y는 ys[t]이다.
    
    # xs[t] -> RNN -> ys[t]
    # RNN은 hs[t]로 이루어져 있다고 생각. -> hs[0]~hs[t]까지 조합.
    
    
    # 여기까지 과정을 살펴보면, hs[t]를 먼저, hs[t-1]과 xs[t]를 이용해서 업데이트를 해주고,
    # 업데이트된 hs[t]를 이용해서 ys[t]를 구한다.
    
    # ※ 주의할점 ※
    # 같은 함수에 대해 같은 입력이 매번 똑같이 사용된다!
    # step이 다ㅡ러다라도, 같은 함수에 대해서는 같은 입력이 사용된다.
    
    # h_0   h_1   h_2   h_3   ...   h_t 
    #  A  -> A  -> A  -> A    ... -> A    
    # x_0   x_1   x_2   x_3   ...   x_t    
    
    # 이 그림으로 치면, h_0에는 x_0만이 들어간다.
    
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # 다음 문자 확률
    
    loss += -np.log(ps[t][targets[t],0]) 
    # softmax (cross-entropy loss)
    # softmax로 분류를 해서, loss를 구한다.  ->   파이썬으로 배우는 머신러닝의 교과서 Chapter 6 부분
    # 최대가능도법으로 확률이라서 -를 붙여준다. (책에 나왔던 부분) 
    # -> 회귀 문제에서의 loss와 똑같이 loss가 작을수록 좋게 판단하기 위함  
      
  # backward pass: compute gradients going backwards
  # backpropagation? 
  # forward를 마친 뒤, backward 실행
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why) # zeros_like 뭔지 모르겠음
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [3]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  
  모델에서 int의 순서로 sample을 만든다.
  h는 memory state고, seed_ix는 첫번째 단계의 seed 문자이다.
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [4]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('c:/app/test.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print("data has",data_size,"characters,", vocab_size," unique")
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias

by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print("----\n", txt,"\n ----\n")

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print("iter", n, "loss: ", smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 155201 characters, 75  unique
----
 f:gSm9(9“JU;o’DII:okrIiJNP?pL)oJaGwKbM‘jDVP’K’B6JmqWAPJLkx2eHB:-BprIW‘,3eßdW?urD0f“sdcC“ßm
UoyqBfRwAdBqdn9QdD;2Csg:vDPq4oSEzß,2oRIfqU,L)p2WOR
VPEhx8;RAAp-4M’MßQKVx6IByO1y6:b;)gd Kg“,.sG,9h1LL2Il?e9ZFG 
 ----

iter 0 loss:  107.93720527471804
----
 eiceSumnFs e elau.iz ’ h    hruw fmaSliure, wsn hrsklhn hftsgSahworrmenhdsksl ewalrerbhug,eeennnn hhu,t msSOhrmuiie eriR hehwbu ir Si wrn er nfVvlnt ateege Sanetn ue SlheduogwbS seuh hl Seomnacewhhael 
 ----

iter 100 loss:  107.1359960611939
----
 TearuiTdeageietarleA rteleze loe Punid.k!eumuoe  slm eewt. dill?esulci  uwlhddei tele i.theeJnnen.ab.a ndeenPhne slt l u sk ynefrhinwdllgrnudrned. lhsngttdcasis nu,  ie h iigeEcirn gbEeßr wicesn Psuei 
 ----

iter 200 loss:  104.73458048107702
----
 sn2fls,srte,2
ner2 ene2
nPe2rw,d2 nsu2fs
 2Lnf 2
rmo2iecg2rnr
2feSu21onnkrlie.eEinei atl
oZn2 uk.2eKu 2klce2lds 2 Eee2enelß V u2lwmf2cher2eldg2ad
,2ialo2f kf2lkf?W hat2
asu2
r lh
nru2nrto2lris2mteahr  
 ----


----
 ick, ekaune undtin, im miestino ?, at umper ugeit orpger Gere subten un arfler, kliau S? stalfen, es zinh Ar Tie ge“s isotnuchwuchWch zaslsie hronenknede eine uruhsch sius Bunen ?Worlen Schwettin Hur  
 ----

iter 3300 loss:  59.98203328777578
----
 ens Er de. ubespanf.
“


Dar dolen,mehtenkchklen dar walneng safhleitlatgabt, Am und, den iephamren, zurn.

Schwas jan die Ler sas und er die ir hir fleiche, Tigen Wedenvichteint! Eibes Waß werallen,  
 ----

iter 3400 loss:  59.574368233997994
----
  kumcht soben koch, eineng melme unhe schab wacht uDnen Zchbedideg, nuliglie, ve Heger Eis homk Zietkem bur jietti abse sauf lims besonge? Iin mie Gersunter erd eicht eie desns um hits ahbein den eich 
 ----

iter 3500 loss:  59.00253797458499
----
 fenker mes espomt Irtrn er Bichen olle dahe mer.“?
?“ich ernkliend. Gem.e bire fie!.
Janr nie ucheichie zesite eire Hich ach, ach Bruglen Bratch. Ditm nimener,. ?3aß und in ?“ Zalbock. Ich olter, sete 
 ----

iter 3600 loss:  58.44907325997964

----
 verichen vagen und Wane Baß sas ie in nicht auf vewoße ? Tlabeck den ver sovuncklie kla jeiufter luze kelrzeisei, andchen, ar ?“ der Wer von den in weih einenkogr ungen in alin andich jeteden zumnisch 
 ----

iter 6600 loss:  53.38827752860374
----
 ahrepicht sienst Dineste hiten spiesarel, vauz, unb elk aum Hedschten baurdscher gratan Kames Trsiinen die Mich hasor die Jandeles, die noch dus dar krerent, den ar deen veher, gah er Ib air ibes gide 
 ----

iter 6700 loss:  53.202984809001705
----
 ller Orachenubie sas os ohrer mie saß die schoß in veresiese. verzfrank. Und beit ichin. Ebounner um deichesreden.
Schite Ledenklachen Gaten Wur Fie kalt.

Jielen, wichemalachtwie Siefe scherrlat scho 
 ----

iter 6800 loss:  53.14369800087369
----
 e werhag Leichang Tid les. Biseßenf vor dir im er sehrmiezt er gat. Ban dar ab ein KAnndal dur er Zomstige die Buhenveptarfen guste ihren raht war Olicheis ? Nieoklarste Jarfuß dedpberger Tollne geizt 
 ----

iter 6900 loss:  53.24379585737242

----
  er fersm in de.

Sies! Im dur vor geienzt! Dan geich waß Bert, Heure ihs die rachtgrar Schwendler eine in wielchanigzt .
Ner togen sis Beran Trei eissen K bosßt  Nich die haun Sie Sie fosprabe, ?Ich  
 ----

iter 9900 loss:  50.123101603040226
----
 ß Unt ier lum. 
Brerlon, dier.

Sie, und wiegen sich der Lidisklelrel furbesen Ku werwerulde guuter boshafle sich ?““ wiese nachs Handlichg Schwarte, memn. ? sich michereht Bese! Sielt: EnQichter Tret 
 ----

iter 10000 loss:  49.99340592526079
----
 mahen und speich kunkrund, er in sein mich Zugof, ogosich im alfes wiuß under au vehwer vom geite Siev wamt meuse und rus jauttei Schren siucht ond ibe. Der gewit Ledenklee ewinmtter. Iche ma?

sachze 
 ----

iter 10100 loss:  49.89308576985726
----
 llehn. ?P wachtr miten schsen er Har Elm getaufren Ersaltte serttertenuvehon weabehe us ahre als zu laon. Er und hranter wandes. Ihral? luiben uber, seiten ha macht hach nur wie de kac klathe Gaste jo 
 ----

iter 10200 loss:  49.80840810021

----
 Ansiebes schweicht einen Ariens und hutz er, ich speden Sieten St wie er dasch sicht bischezent, die Pasend gißssalchhrandeter die neete Wediel wun absqestife den, sicht haft Rrote!“

And daß welzen e 
 ----

iter 13200 loss:  50.082368570976485
----
  , die Schwedenk?

Sihrpauge sergenten, wunk waldeht, Hicht se laltatehckebe Sawand dam mahn ein dach eingafgrorbelen wondo,relle wortwunn sichsie eit nate stits Zau aZt ond fur speife, wur Kuten“ Bru 
 ----

iter 13300 loss:  49.96897040546916
----
 edend dieler hin ersuge ihm dehn, werdenstus, ernopgerkannist. Ein derdenelutscheren klanken Zickteaesih.

Ich dreplas Korten Erfante gaum Unteimleis beriste Er buflalipstige zumwautter, gaugen Wedeß  
 ----

iter 13400 loss:  49.98046872095989
----
 inen die, ?Sie voht mich Peralt Reicht, on es eahe sticht, ebacht. Was sie wum werklichte, zu Eine Spagand fie leich vom zurgsichte ich nervem und rahlit. ?Erigen, wie ? dies sied spdiersimfe watsusst 
 ----

iter 13500 loss:  49.9647834168

...

----
  allenn ses, Roginant vor igter es won vollzlichzen ea, and das Schrelbente.“

Zwor des zu so das Biodosagts wank auf den Brquflie, vertschodlichte haten, eine Pfrak stlaßtel dar Raussalligst er empsc 
 ----

iter 138600 loss:  43.01471547286494
----
 n Mauben zuruplere mitscheer brwamabche, nicht geunbre. SchwizumaSen!“

Gepiehengrussanigen schrockschoff spumete eine Gakschofzeier Kaus in den Ihne heiteifmehst, .
?Umig, mit geschief, beumppor gess 
 ----

iter 138700 loss:  42.782714699814335
----
 y.

Num ein mit der Erscho betatige leisen Kirbebraldsin Spiedenkeippelgetet geuuchtutend Vurgalgemschtamichen, schnieren.

?Unbehramm gotir, einen Nachts ungbaste, verobonftruchen Bilter mit spiel Re 
 ----

iter 138800 loss:  42.77493663533839
----
 ch zwein gindem farben, die Gede, er hatte hattigem Erandaugerben Nate gungere, zu wahte er mit den im sollen die gewohmen Wocheie, einen, aug under. In dem Sirzevorand geschreingung!“ in diese, wunkl 
 ----

iter 138900 loss:  42.700951

----
 e mitwerrantsnerschaftzchen ließ er noch auf, wunder pessanste Blanm blocken in Ihlette Schacher, ich man undenihen war Sogallen, daß und von gewunge, unschene sillelde“ Geglesten ? in Pofar betie, di 
 ----

iter 141900 loss:  41.74538506509782
----
 gefblitt, daß Glank ihre Ahrarfe. Noten ließ ich Aust sein Temschen sie erst sich ich sich besahsen meit Blanhamlacht Gelihmigtrochtweiehstauch losite so seine Blank sich fase, daß er fiecht. Verdamme 
 ----

iter 142000 loss:  41.890255388254026
----
 in Weisfendweisen. Schwespager aber viebseif Achs einen Gesichter kam Frablent mein Narmschanben holtstenungezlagtkor, jedegunigun der Lachegte Zwaren Hussen Tog, er sinhen von igreitte alf.

Siellied 
 ----

iter 142100 loss:  42.14169442521103
----
 h dem Gastliche, IhlerFkeilen. Diesgeruginden. Einen hewandbende stragbenden Arserelm Luch zwausche, muh flobelussehendisste, plauen Zieshawordlichen Geralln nicht, daß Schwedenklei einen Frech beschl 
 ----

iter 142200 loss:  42.297779

----
 enden, Sieb ginfseitze Unvonde“n.

Au diesen der Teglund auf seine Reisen Luucheiel. Verardigsen einem ?Wunse die ihm und auf ihn beschlaschmen sannele vieren Behrnuche? Der arf Lachte, eine jerzen!“
 
 ----

iter 145200 loss:  42.73276259061406
----
 Fllasis nicht verflacht, hogeres Attissen Wirkeren ? und aufdiften, fleei unglie zuflichteist. Mar zurschrueden. Und von eine Auslebruck!

Schwiderken! Limmen haren Storuntilliger schwerkenze zu Nucht 
 ----

iter 145300 loss:  43.051267047864926
----
 rzwas errold mit eit daren verenbiet und sich in sehaste do kunenden Berzt mich einem richendren verdlundend sal geter, nacht.

Banstler? Gesie und wermen, doch eingesbel ihr zweisen, augaltsein sich  
 ----

iter 145400 loss:  43.04292090441508
----
 gawolge. Er allee, in das Verrotz uber berut der funn wannen Schraszeenschlankusterkuaren, hatte diech daß er mezunten ? den Kotzei. Es wurde, elrangen. Kill. Schwedenklee was end um von Errrich. Der  
 ----

iter 145500 loss:  42.926094

----
 das Frunhiggen. Er nur sie wan Austbichte mit noch erkolien furssalle durummenkkenlte, zu nicht im ? zurund auch einer Sie luß. Selut noch versans bleidem Blacke Fraalt weig, die und beicht liesen Gel 
 ----

iter 148500 loss:  42.35684999434117
----
  wieder uberbesag urstande, die Sie ?ich war funder Hinges noch segtreinen er, bewehrlich nich er!“ verwahzuckte, und der Schwedaumles Glankschneite Nachhegten ein gelistis. Eill. Im Kochen, aber er d 
 ----

iter 148600 loss:  42.31034444180034
----
 
?Die Plenze ander Gesagen Denor bishichtie, ius und damer, als in der in Pelche.

?Ich sie ist der aufvor ich mit intwedre haben Zugervallite Bitzig. Sie um Berr soguran, unmor ...

Schwedenklee, gef 
 ----

iter 148700 loss:  42.34120456344564
----
 en Fischindig er es nohnen fatend dapwosorung mitrutgtruhzen Rurt, sahm.

?Jar Hennetyipchechse! Er in ei schriichen, wahlein, das schweiltalet, dem Habiche den Onal dast der eine Latte sie du uufsche 
 ----

iter 148800 loss:  42.4888635

KeyboardInterrupt: 